In [ ]:
from keras.datasets import mnist
import numpy as np
from scipy.optimize import minimize


(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


train_images = train_images.reshape((60000, 28 * 28)).astype(np.float32) / 255
test_images = test_images.reshape((10000, 28 * 28)).astype(np.float32) / 255

print(f'Training images shape: {train_images.shape}')
print(f'Training labels shape: {train_labels.shape}')
print(f'Test images shape: {test_images.shape}')
print(f'Test labels shape: {test_labels.shape}')


Training images shape: (60000, 784)
Training labels shape: (60000,)
Test images shape: (10000, 784)
Test labels shape: (10000,)


In [13]:
def pca(X, num_components):
    X_meaned = X - np.mean(X, axis=0)
    covariance_matrix = np.cov(X_meaned, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    sorted_index = np.argsort(eigenvalues)[::-1]
    sorted_eigenvectors = eigenvectors[:, sorted_index]
    eigenvector_subset = sorted_eigenvectors[:, 0:num_components]
    X_reduced = np.dot(X_meaned, eigenvector_subset)
    return X_reduced



num_components = 50

subset_size = 1000
train_images_subset = train_images[:subset_size]
train_labels_subset = train_labels[:subset_size]
test_images_subset = test_images[:subset_size]
test_labels_subset = test_labels[:subset_size]
X_train_reduced = pca(train_images_subset, num_components)
X_test_reduced = pca(test_images_subset, num_components)

print(f'Reduced training data shape: {X_train_reduced.shape}')
print(f'Reduced test data shape: {X_test_reduced.shape}')

print(f'Training labels shape: {train_labels_subset.shape}')
print(f'Test images shape: {test_images_subset.shape}')
print(f'Test labels shape: {test_labels_subset.shape}')


Reduced training data shape: (1000, 50)
Reduced test data shape: (1000, 50)
Training labels shape: (1000,)
Test images shape: (1000, 784)
Test labels shape: (1000,)


In [ ]:
class SVM:
    def __init__(self, C=1.0, gamma=1.0):
        self.C = C
        self.gamma = gamma
        self.alpha = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.b = 0

    def kernel(self, x1, x2):
        return np.exp(-self.gamma * np.linalg.norm(x1 - x2) ** 2)

    def fit(self, X, y):
        n_samples = len(X)
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i, j] = self.kernel(X[i], X[j])
        
        # Objective function coefficients
        P = K * np.outer(y, y)
        q = -np.ones(n_samples)
        
        # Constraints
        A = y.astype(float)
        b = 0.0
        G = np.vstack((-np.eye(n_samples), np.eye(n_samples)))
        h = np.hstack((np.zeros(n_samples), np.ones(n_samples) * self.C))
        
        # Solve the quadratic program
        def objective(alpha):
            return 0.5 * np.dot(alpha.T, np.dot(P, alpha)) + np.dot(q.T, alpha)
        
        constraints = ({'type': 'eq', 'fun': lambda alpha: np.dot(A, alpha) - b},
                    {'type': 'ineq', 'fun': lambda alpha: alpha},
                    {'type': 'ineq', 'fun': lambda alpha: self.C - alpha})
        
        bounds = [(0, self.C) for _ in range(n_samples)]
        
        res = minimize(objective, np.zeros(n_samples), method='SLSQP', constraints=constraints, bounds=bounds)
        
        self.alpha = res.x
        
        sv = self.alpha > 1e-5
        self.support_vectors = np.array(X)[sv]
        self.support_vector_labels = y[sv]
        self.alpha = self.alpha[sv]
        
        self.b = np.mean([y_k - self.predict_single(x_k) for (x_k, y_k) in zip(self.support_vectors, self.support_vector_labels)])

    def predict_single(self, x):
        result = self.b
        for alpha_k, x_k, y_k in zip(self.alpha, self.support_vectors, self.support_vector_labels):
            result += alpha_k * y_k * self.kernel(x_k, x)
        return np.sign(result)

    def predict(self, X):
        return np.array([self.predict_single(x) for x in X])

def prepare_labels(y, class_label):
    return np.where(y == class_label, 1, -1)

In [20]:
svm_classifiers = []
n_classes = 10

for class_label in range(n_classes):
    print(f'Training classifier for digit {class_label}...')
    y_train_binary = prepare_labels(train_labels_subset, class_label)
    svm = SVM(C=1.0, gamma=0.05)
    svm.fit(X_train_reduced, y_train_binary)
    svm_classifiers.append(svm)

def predict_multi_class(X):
    predictions = np.zeros((X.shape[0], n_classes))
    for idx, svm in enumerate(svm_classifiers):
        preds = svm.predict(X)
        predictions[:, idx] = preds
    return np.argmax(predictions, axis=1)

y_pred = predict_multi_class(X_test_reduced)
accuracy = np.mean(y_pred == test_labels_subset)
print(f'Test set accuracy: {accuracy * 100:.2f}%')

Training classifier for digit 0...
Training classifier for digit 1...
Training classifier for digit 2...
Training classifier for digit 3...
Training classifier for digit 4...
Training classifier for digit 5...
Training classifier for digit 6...
Training classifier for digit 7...
Training classifier for digit 8...
Training classifier for digit 9...
Test set accuracy: 8.50%
